# 인공지능 스피커

In [11]:
# 환경변수 불러오기
from dotenv import load_dotenv

load_dotenv()

True

## 인공지능 챗봇

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

### 1) 구성 요소 생성

#### Model

In [13]:
# Model 생성
model = ChatOpenAI(
    model="gpt-4o-mini",  # 비용 gpt-4.1-nano < gpt-4o-mini < gpt-4
    temperature=0.5       # 창의성 정도(0~1 사이의 값. 1에 가까울수록 창의적인 답변)
)

#### Prompt

In [14]:
system_prompt = "당신은 친절하고 유익한 AI 어시스턴트입니다. 사용자의 질문에 대해 정확하고 도움이 되는 답변을 제공하세요."

# 히스토리를 반영한 Prompt 생성
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{input}")
])

#### OutputParser

In [15]:
# OutputParser 생성
output_parser = StrOutputParser()

### 2) Chain 만들기

In [16]:
# Runnable 생성
runnable = prompt | model | output_parser 

In [17]:
# 히스토리가 저장될 변수 생성
store = {}  

# 세션 ID별 히스토리를 가져오는 함수
def get_session_history(session_id):
	# store의 key에 session_id가 없는 경우 session_id를 key에 추가
    if session_id not in store: 
        store[session_id] = ChatMessageHistory()
        
    # session_id 키의 값을 반환
    return store[session_id]

In [18]:
# Chain 생성
chain = (
    RunnableWithMessageHistory(          # RunnableWithMessageHistory 객체 생성
        runnable,                        # 실행할 Runnable 객체
        get_session_history,             # 세션 ID별 히스토리를 가져오는 함수
        input_messages_key="input",      # 입력 메시지의 키
        history_messages_key="history",  # 기록 메시지의 키
    )
)

### 챗봇 테스트

In [19]:
# 무한 루프
while True:
    ## 사용자 입력 받기
    question = input("[USER] ")
    print(f"[USER]: {question}")
    ## 종료 조건
    if question == "q":
        print("대화를 종료합니다.")
        break
    ## 응답 받기
    response = chain.invoke(
        {"input": question},
        {"configurable": {"session_id": "user1"}}
    )
    print(f"[AI]: {response}")

[USER]: 안녕!


APIConnectionError: Connection error.

## 스피커 1 (TTS: 기본 제공)

In [10]:
import os
from openai import OpenAI
import speech_recognition as sr
import tempfile
from playsound import playsound
from pathlib import Path
import time
from pydub import AudioSegment

# OpenAI 클라이언트 생성
client = OpenAI()
r = sr.Recognizer()

print("📢 인공지능 스피커 시작!")

while True:
    try:
        with sr.Microphone() as source:
            print("\n🎤 듣는 중...")
            r.adjust_for_ambient_noise(source)
            audio = r.listen(source)

            print("🧠 인식 중...")
            question = r.recognize_openai(audio)
            print("📝 인식된 텍스트:", question)

            if question.strip() == "종료":
                print("\n📢 인공지능 스피커 종료!")
                break

            # AI 응답 생성
            response_text = chain.invoke(
                {"input": question},
                {"configurable": {"session_id": "user1"}}
            )
            print("🤖 챗봇 답변:", response_text)

            # TTS 응답(MP3)을 임시로 저장
            with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as mp3_file:
                mp3_path = Path(mp3_file.name)
                with client.audio.speech.with_streaming_response.create(
                    model="gpt-4o-mini-tts",
                    voice="coral",
                    input=response_text,
                    instructions="굵은 남자 목소리"
                ) as tts_response:
                    tts_response.stream_to_file(str(mp3_path))

            # MP3 → WAV 변환 (PCM 16bit)
            wav_path = mp3_path.with_suffix(".wav")
            audio_segment = AudioSegment.from_mp3(str(mp3_path))
            audio_segment.export(str(wav_path), format="wav", parameters=["-acodec", "pcm_s16le"])

            # 약간 대기 후 재생
            time.sleep(0.2)
            if wav_path.exists():
                print("🔊 재생 중...")
                playsound(str(wav_path))
                os.remove(wav_path)
                os.remove(mp3_path)
            else:
                print(f"❌ 파일이 존재하지 않습니다: {wav_path}")

    except KeyboardInterrupt:
        print("\n🛑 수동 종료")
        break

    except Exception as e:
        print(f"🚨 오류 발생: {e}")

📢 인공지능 스피커 시작!

🎤 듣는 중...
🧠 인식 중...
📝 인식된 텍스트: チャンネル登録をお願いいたします。
🤖 챗봇 답변: チャンネル登録を促すメッセージですね！動画やコンテンツを作成している場合、視聴者にチャンネル登録をお願いすることはとても大切です。具体的には、次のような言葉を使うと良いでしょう：

「この動画が気に入ったら、ぜひチャンネル登録をお願いします！新しいコンテンツをいち早くお届けしますので、お見逃しなく！」

視聴者にとってのメリットを伝えると、登録してもらいやすくなりますよ。頑張ってください！
🔊 재생 중...
🚨 오류 발생: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\lunia\\AppData\\Local\\Temp\\tmpl7ypi9oi.wav'

🎤 듣는 중...

🛑 수동 종료


## 스피커 2 (TTS: 보이스 클로닝)

In [20]:
import os 
from openai import OpenAI
from elevenlabs.client import ElevenLabs
from elevenlabs import play
import speech_recognition as sr

# OpenAI 클라이언트 생성
client = OpenAI()

# ElevenLabs 클라이언트 생성
client = ElevenLabs(api_key=os.getenv("ELEVENLABS_API_KEY"))

# 음성인식 도구 생성
r = sr.Recognizer()

print("📢 인공지능 스피커 시작!")

while True:
    try:
        with sr.Microphone() as source:
            print("\n🎤 듣는 중...")
            # STEP 1: 마이크로부터 입력
            r.adjust_for_ambient_noise(source)  # 주변 소음 조정
            audio = r.listen(source)
            
            print("🧠 인식 중...")

            # STEP 2: Whisper API로 텍스트 변환
            question = r.recognize_openai(audio)
            print("📝 인식된 텍스트:", question)

            ## 종료 조건
            if question == "종료":
                print("\n📢 인공지능 스피커 종료!")
                break

            # STEP 3: 인공지능 챗봇 응답
            response = chain.invoke(
                {"input": question},
                {"configurable": {"session_id": "user1"}}
            )
            print("🤖 챗봇 답변:", response)

            # STEP 4: TTS로 음성 출력
            audio = client.text_to_speech.convert(
                text=response,
                voice_id="S8IIf77q99OgYuPS5P9n",
                model_id="eleven_multilingual_v2",
                output_format="mp3_44100_128",
                voice_settings={
                    "stability": 0.3,
                    "similarity_boost": 0.8,
                    "use_speaker_boost": True
                }
            )

            # 음성 재생
            play(audio)

    except KeyboardInterrupt:
        print("\n🛑 수동 종료")
        break

📢 인공지능 스피커 시작!

🎤 듣는 중...
🧠 인식 중...


APIConnectionError: Connection error.